<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/Assignment4/k_data_augmentation_classification__text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow import keras
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 0s 0us/step


preprocess the data by padding the sequences to a uniform length and creating a validation set

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 200
train_data = pad_sequences(train_data, maxlen=maxlen)
test_data = pad_sequences(test_data, maxlen=maxlen)

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]


define the model using an embedding layer, a global max pooling layer, and two dense layers with dropout regularization

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalMaxPooling1D, Dense, Dropout

embedding_dim = 16

model = Sequential([
    Embedding(10000, embedding_dim, input_length=maxlen),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Train the model with data augmentation using Keras.
During training, Keras will automatically apply data augmentation techniques like shuffling and random cropping to the input data to prevent overfitting



In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=40,
    batch_size=512,
    validation_data=(x_val, y_val)
)


In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 1s 2ms/step - loss: 0.5842 - accuracy: 0.8201
[0.5842366218566895, 0.8200799822807312]


Data augmentation using nlpaug, replacing words with synonyms

In [1]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.7 MB/s eta 0:00:00


In [14]:
import nlpaug.augmenter.word as naw

# Define the augmentation function
def augment_text(text, aug):
    augmented_text = aug.augment(text)
    return augmented_text

# Define the augmentation method
aug = naw.SynonymAug(aug_src='wordnet')


# Augment the text data
augmented_x_train = [aug.augment(str(text)) for text in partial_x_train]
augmented_x_val = [aug.augment(str(text)) for text in x_val]
augmented_test_data = [aug.augment(str(text)) for text in test_data]


In [15]:
# Convert the augmented text data into sequences of integers
from keras.preprocessing.text import Tokenizer

# Create a tokenizer object
tokenizer = Tokenizer(num_words=10000)
augmented_x_train = tokenizer.texts_to_sequences(augmented_x_train)
augmented_x_val = tokenizer.texts_to_sequences(augmented_x_val)
augmented_test_data = tokenizer.texts_to_sequences(augmented_test_data)

# Pad the augmented data
augmented_x_train = pad_sequences(augmented_x_train, maxlen=maxlen)
augmented_x_val = pad_sequences(augmented_x_val, maxlen=maxlen)
augmented_test_data = pad_sequences(augmented_test_data, maxlen=maxlen)


In [16]:
# Train the model with augmented data
history = model.fit(
    augmented_x_train,
    partial_y_train,
    epochs=40,
    batch_size=512,
    validation_data=(augmented_x_val, y_val)
)

# Evaluate the model with the augmented test data
results = model.evaluate(augmented_test_data, test_labels)
print(results)

Epoch 1/40
30/30 [==============================] - 2s 32ms/step - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6933 - val_accuracy: 0.4947
Epoch 2/40
30/30 [==============================] - 1s 25ms/step - loss: 0.6932 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 3/40
30/30 [==============================] - 1s 30ms/step - loss: 0.6931 - accuracy: 0.5046 - val_loss: 0.6934 - val_accuracy: 0.4947
Epoch 4/40
30/30 [==============================] - 1s 34ms/step - loss: 0.6932 - accuracy: 0.5017 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 5/40
30/30 [==============================] - 1s 33ms/step - loss: 0.6932 - accuracy: 0.5001 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 6/40
30/30 [==============================] - 1s 32ms/step - loss: 0.6933 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 7/40
30/30 [==============================] - 1s 33ms/step - loss: 0.6932 - accuracy: 0.4942 - val_loss: 0.6932 - val_accuracy: 0.4947
Epoch 8/40
30

With data augmentation(replacing words with synonyms), accuracy reduced from 82% to 50%